In [1]:
import pandas as pd
import numpy as np

In [2]:
from pandas import read_csv
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Bidirectional, Reshape
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math
import matplotlib.pyplot as plt

In [3]:
#run
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from sklearn import metrics
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df = pd.read_csv('/content/drive/MyDrive/audio-interspeech/2075_concatenate_dm_4featu.csv')
df = df.drop(['Unnamed: 0'], axis = 1)
df.shape

(2075, 181)

In [6]:
np.random.seed(42)
siz=415
df_read = df.copy()
df1 = df_read.sample(siz)
df_read = df_read.drop(df1.index)
df2 = df_read.sample(siz)
df_read = df_read.drop(df2.index)
df3 = df_read.sample(siz)
df_read = df_read.drop(df3.index)
df4 = df_read.sample(siz)
df_read = df_read.drop(df4.index)
df5 = df_read.copy()

print(df1.shape)
print(df2.shape)
print(df3.shape)
print(df4.shape)
print(df5.shape)

(415, 181)
(415, 181)
(415, 181)
(415, 181)
(415, 181)


In [7]:
q = list(df1.index)+list(df2.index)+list(df3.index)+list(df4.index)+list(df5.index)
print(df1.index)

Int64Index([ 881,  453, 2004, 1353,  281,  941, 1185, 1159, 1138,  599,
            ...
             834, 1730,  353, 1345, 1190, 1375,  185,  701, 1671, 1982],
           dtype='int64', length=415)


In [8]:
train = pd.concat([df1,df2,df3,df4], axis=0)
test = df5.copy()

Y_train = np.array(train['class'])
X_train= np.array(train.drop(['class'],axis=1))
# X_train=X_train.reshape(X_train.shape[0], 1 , X_train.shape[1])

Y_val=np.array(test['class'])
X_val = np.array(test.drop(['class'],axis=1))
# X_val=X_val.reshape(X_val.shape[0], 1 , X_val.shape[1])

In [9]:
TIME_STEPS= X_train.shape[1]

In [10]:
#run
def scaled_dot_product_attention(query, key, value, mask):
  matmul_qk = tf.matmul(query, key, transpose_b=True)

  depth = tf.cast(tf.shape(key)[-1], tf.float32) # scaler value
  logits = matmul_qk / tf.math.sqrt(depth)

  # add the mask zero out padding tokens.
  if mask is not None:
    logits += (mask * -1e9)

  attention_weights = tf.nn.softmax(logits, axis=-1)

  return tf.matmul(attention_weights, value)

In [11]:
#run
# This allows to the transformer to know where there is real data and where it is padded
def create_padding_mask(seq):
  seq = tf.cast(tf.math.equal(seq, 0), tf.float32) # check if a values of the sequence are 0 or not

  # add extra dimensions to add the padding
  # to the attention logits.
  return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

In [12]:
#run
class MultiHeadAttention(tf.keras.layers.Layer):

  def __init__(self, d_model, num_heads, name="multi_head_attention"):
    super(MultiHeadAttention, self).__init__(name=name)
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.query_dense = tf.keras.layers.Dense(units=d_model)
    self.key_dense = tf.keras.layers.Dense(units=d_model)
    self.value_dense = tf.keras.layers.Dense(units=d_model)

    self.dense = tf.keras.layers.Dense(units=d_model)

  def split_heads(self, inputs, batch_size):
    inputs = tf.reshape(
        inputs, shape=(batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(inputs, perm=[0, 2, 1, 3])

  def call(self, inputs):
    query, key, value, mask = inputs['query'], inputs['key'], inputs[
        'value'], inputs['mask']
    batch_size = tf.shape(query)[0]

    # linear layers
    query = self.query_dense(query)
    key = self.key_dense(key)
    value = self.value_dense(value)

    # split heads
    query = self.split_heads(query, batch_size)
    key = self.split_heads(key, batch_size)
    value = self.split_heads(value, batch_size)

    scaled_attention = scaled_dot_product_attention(query, key, value, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))

    outputs = self.dense(concat_attention)

    return outputs

In [13]:
#run
class PositionalEncoding(tf.keras.layers.Layer):

  def __init__(self, position, d_model):
    super(PositionalEncoding, self).__init__()
    self.pos_encoding = self.positional_encoding(position, d_model)

  def get_angles(self, position, i, d_model):
    angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
    return position * angles

  def positional_encoding(self, position, d_model):
    angle_rads = self.get_angles(
        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model=d_model)
    # apply sin to even index in the array
    sines = tf.math.sin(angle_rads[:, 0::2])
    # apply cos to odd index in the array
    cosines = tf.math.cos(angle_rads[:, 1::2])

    pos_encoding = tf.concat([sines, cosines], axis=-1)
    pos_encoding = pos_encoding[tf.newaxis, ...]
    return tf.cast(pos_encoding, tf.float32)

  def call(self, inputs):
    return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

In [14]:
#run
def encoder_layer(units, d_model, num_heads, dropout,name="encoder_layer"):
  inputs = tf.keras.Input(shape=(None,d_model ), name="inputs")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': padding_mask
      })
  attention = tf.keras.layers.Dropout(rate=dropout)(attention)
  attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
  # outputs = tf.keras.layers.Dropout(rate=dropout)(outputs) # extra layer added
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [15]:
#run
def encoder(time_steps,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            projection,
            name="encoder"):
  inputs = tf.keras.Input(shape=(None,d_model), name="inputs")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  if projection=='linear':
    ## We implement a linear projection based on Very Deep Self-Attention Networks for End-to-End Speech Recognition. Retrieved from https://arxiv.org/abs/1904.13377
    projection=tf.keras.layers.Dense( d_model,use_bias=True, activation='linear')(inputs)
    print('linear')

  else:
    projection=tf.identity(inputs)
    print('none')

  projection *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  projection = PositionalEncoding(time_steps, d_model)(projection)

  outputs = tf.keras.layers.Dropout(rate=dropout)(projection)

  for i in range(num_layers):
    outputs = encoder_layer(
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        name="encoder_layer_{}".format(i),
    )([outputs, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [16]:
#run
def transformer_BiLSTM(time_steps,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                output_size,
                projection,
                name="transformer"):
  inputs = tf.keras.Input(shape=(None,d_model), name="inputs")

  enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(tf.dtypes.cast(

      #Like our input has a dimension of length X d_model but the masking is applied to a vector
      # We get the sum for each row and result is a vector. So, if result is 0 it is because in that position was masked
      tf.math.reduce_sum(
      inputs,
      axis=2,
      keepdims=False,
      name=None
  ), tf.int32))

  enc_outputs = encoder(
      time_steps=time_steps,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
      projection=projection,
      name='encoder'
  )(inputs=[inputs, enc_padding_mask])

  #We reshape for feeding our FC in the next step
  outputs=tf.reshape(enc_outputs,(-1,time_steps*d_model))

  #We predict our class
  # model.add(Reshape((input_shape[0],1), input_shape=input_shape))
  outputs = tf.keras.layers.Reshape((d_model,1))(outputs)
  outputs = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(d_model))(outputs)

  outputs = tf.keras.layers.Dense(units=d_model/2,use_bias=True, name="outputs_1")(outputs)
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.Dense(units=d_model/4,use_bias=True, name="outputs_2")(outputs)
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.Dense(units=output_size,use_bias=True, name="outputs_3")(outputs)

  return tf.keras.Model(inputs=[inputs], outputs=outputs, name='audio_class')

In [17]:
# def create_BiLSTM(hidden_units, dense_units, input_shape, activation):
#     model = Sequential()
#     # model.add(LSTM(hidden_units,input_shape=input_shape))
#     model.add(Reshape((input_shape[0],1), input_shape=input_shape))
#     model.add(Bidirectional(LSTM(hidden_units)))
#     model.add(Dense(units = input_shape[0]//2, activation='relu'))
#     model.add(Dropout(0.5))
#     model.add(Dense(units=input_shape[0]//4, activation='relu'))
#     model.add(Dropout(0.5))
#     model.add(Dense(units=1))
#     model.compile(loss='mean_absolute_error', optimizer='adam')
#     return model

In [18]:
# model = create_BiLSTM(hidden_units=180, dense_units=1, input_shape=(TIME_STEPS,1),
#                    activation=['tanh', 'tanh'])
# model.fit(X_train, Y_train, epochs=1000, batch_size=64, verbose=2,validation_data=(X_val, Y_val))

In [19]:
#run call the transformer model
#df_read = df.copy()
#df_read=(df_read-df_read.mean())/df_read.std()
from sklearn.model_selection import train_test_split

#train, test = train_test_split(df_read, test_size=0.2)

train = pd.concat([df1,df2,df3,df4], axis=0)
test = df5.copy()

Y_train = np.array(train['class'])
X_train= np.array(train.drop(['class'],axis=1))
X_train=X_train.reshape(X_train.shape[0], 1 , X_train.shape[1])

Y_val=np.array(test['class'])
X_val = np.array(test.drop(['class'],axis=1))
X_val=X_val.reshape(X_val.shape[0], 1 , X_val.shape[1])
NUM_LAYERS =  1

D_MODEL = X_train.shape[2]
NUM_HEADS =  4
UNITS =  2048
DROPOUT = 0.1 #0.1
TIME_STEPS= X_train.shape[1]
OUTPUT_SIZE=1
batch_size=64
print(D_MODEL)
model = transformer_BiLSTM(time_steps=TIME_STEPS,
  num_layers=NUM_LAYERS,
  units=UNITS,
  d_model=D_MODEL,
  num_heads=NUM_HEADS,
  dropout=DROPOUT,
  output_size=OUTPUT_SIZE,
  projection='linear')

#run
# model.compile(optimizer=tf.keras.optimizers.Adam(0.00005), loss='mae') #org
model.compile(optimizer=tf.keras.optimizers.Adam(0.000003), loss='mse')
# model.compile(optimizer=tf.keras.optimizers.SGD(0.01), loss='mae')
#
# callback = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience=80, restore_best_weights=True)
# history = model.fit(X_train,Y_train, epochs=1000, validation_data=(X_val, Y_val), callbacks=[callback])
history = model.fit(X_train,Y_train, epochs=1000, validation_data=(X_val, Y_val))

180
linear
Epoch 1/1000
52/52 [==============================] - 16s 45ms/step - loss: 10.5074 - val_loss: 9.4969
Epoch 2/1000
52/52 [==============================] - 1s 23ms/step - loss: 9.9129 - val_loss: 8.6862
Epoch 3/1000
52/52 [==============================] - 1s 26ms/step - loss: 8.9548 - val_loss: 7.4738
Epoch 4/1000
52/52 [==============================] - 1s 23ms/step - loss: 7.6530 - val_loss: 5.9930
Epoch 5/1000
52/52 [==============================] - 1s 23ms/step - loss: 6.1572 - val_loss: 4.5137
Epoch 6/1000
52/52 [==============================] - 1s 23ms/step - loss: 4.7327 - val_loss: 3.2909
Epoch 7/1000
52/52 [==============================] - 1s 25ms/step - loss: 3.5929 - val_loss: 2.4580
Epoch 8/1000
52/52 [==============================] - 2s 32ms/step - loss: 2.7661 - val_loss: 1.9846
Epoch 9/1000
52/52 [==============================] - 2s 33ms/step - loss: 2.2565 - val_loss: 1.7874
Epoch 10/1000
52/52 [==============================] - 2s 34ms/step - loss: 2.

In [22]:
#run
import time
st = time.time()
p1 = np.array(model(X_val)).flatten()
end = time.time()
# print(end, st, len(p1))
print((end-st)/len(p1))
p2 = np.array(model(X_train)).flatten()
print(X_train.shape)
print(X_val.shape)

0.00017973428749176393
(1660, 1, 180)
(415, 1, 180)


In [23]:
from scipy import stats
from scipy.stats import kendalltau
# print(np.corrcoef(p2, Y_train), stats.spearmanr(p2, Y_train), kendalltau(p2,Y_train).correlation)
print(np.corrcoef(p1, Y_val)[1][0], stats.spearmanr(p1, Y_val).correlation, kendalltau(p1,Y_val).correlation)


0.7927345514994997 0.7836971771452386 0.5910736674393636
